<br>

<center><img src="https://www.htu.edu.jo/images/ThumbnailsCoverPhotos/HTU%20Logo-250px.png" alt="HTU"  width="180px" align="center">


<br>

<p>

**Data Mining**

10204310

Section (3)

**Analyze and Implement text and graph mining application**

**Submitted to**

Eng. Bassam Kasasbeh

**Submitted on**

December 23rd, 2023

**Submitted by**

Marwan Tarek Shafiq Al Farah

**Student ID**

21110011

Fall 2023 – 2024
</p></center>

# **Install and Import Libraries**

In [1]:
# Import libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_similarity
import matplotlib.pyplot as plt
import networkx as nx
from bidi.algorithm import get_display
import arabic_reshaper
from nltk.corpus import wordnet
from nltk.stem.isri import ISRIStemmer
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
from tkinter import ttk
import os

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marwa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marwa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marwa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\marwa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# **Loading The Dataset**

In [2]:
path = 'D:/Marwan/HTU/Third Year/Fall Semester/Data Mining/My Solutions/Mid/Code/'

In [3]:
# Load dataset
df_oj = pd.read_excel(path + 'Complaints dataset.xlsx', sheet_name='Complaints_v3')
df_oj = df_oj[df_oj.columns[:-2]]
df_oj = df_oj[df_oj.duplicated() == False]

In [4]:
# Create a copy of the dataset
df = df_oj.copy()

# **Functions to Preprocess the Text**

In [5]:
# Adding spaces between each type of different characters
def add_space_between_characters(text):
    # Unicode range for Arabic characters
    arabic_range = '\u0600-\u06FF'

    # Define patterns for different types of characters
    patterns = [
        # English letter followed by Arabic, number, or special character
        rf'([a-zA-Z])([\d{arabic_range}]|[^\w\s])',
        # Arabic letter followed by English, number, or special character
        rf'([{arabic_range}])([a-zA-Z\d]|[^\w\s])',
        # Number followed by English, Arabic, or special character
        rf'(\d)([a-zA-Z{arabic_range}]|[^\w\s])',
        # Special character followed by English, Arabic, or number
        rf'([^\w\s])([a-zA-Z{arabic_range}\d])'
    ]

    # Function to insert space between matched groups
    def insert_space(match):
        return ' '.join(match.groups())

    # Apply each pattern
    for pattern in patterns:
        text = re.sub(pattern, insert_space, text)

    return text

In [6]:
# Precompile Regular Expressions
normalize_arabic_regex = re.compile(r'[إأٱآا]')
yaa_regex = re.compile(r'[ؤئ]')
haa_regex = re.compile(r'[ة]')
tashkeel_regex = re.compile(r'[ًٌٍَُِّْ]')
alnum_regex = re.compile(r'\w')

# Load Stopwords Once
english_stopwords = set(stopwords.words('english'))
arabic_stopwords = set(stopwords.words('arabic'))

# Lemmatizer Initialization
lemmatizer = WordNetLemmatizer()

# Stemmer Initialization
st = ISRIStemmer()

In [7]:
# Function for POS tagging
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [8]:
# Function to preprocess the text
def clean_text(text, arabic_stemming = False):
    # Normalize English text
    text = text.lower()

    # Normalize Arabic Text
    text = normalize_arabic_regex.sub('ا', text)
    text = yaa_regex.sub('ء', text)
    text = haa_regex.sub('ه', text)
    text = tashkeel_regex.sub('', text)

    # Add Spaces between Arabic Letters, English Letters, Symbols, and Numbers
    text = add_space_between_characters(text)

    # Tokenize the Text
    words = word_tokenize(text)

    # Removing Non-Alphanumeric Characters
    words = [word for word in words if alnum_regex.match(word) and word not in english_stopwords and word not in arabic_stopwords]

    # Combined English Processing: Lemmatization and Stopwords Removal
    words = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in nltk.pos_tag(words)]

    # Arabic Preprocessing
    words = [word[len("ال"):] if word.startswith("ال") else word for word in words]

    # Arabic Stemming
    if arabic_stemming:
      words = [st.stem(word) for word in words]

    # Return the Preprocessed Text
    return " ".join(words)

# **Preprocess the Text & TF-IDF**

In [9]:
# Preprocessing the text
df['CASE_DESC'] = df['CASE_DESC'].apply(clean_text)

In [10]:
# Using TF-IDF
vectorizer = TfidfVectorizer()
tfidf_documents = vectorizer.fit_transform(df['CASE_DESC'])

# **Input GUI**

In [11]:
def display_dataframe(df, treeview):
    # Clear previous data
    for i in treeview.get_children():
        treeview.delete(i)
    
    # Setting up new dataframe structure
    treeview["column"] = list(df.columns)
    treeview["show"] = "headings"
    for column in treeview["column"]:
        treeview.heading(column, text=column)

    # Inserting each row of data into the treeview
    df_rows = df.to_numpy().tolist()
    for row in df_rows:
        treeview.insert("", "end", values=row)

In [12]:
def additional_action1():
    # Create a new window
    new_window = tk.Toplevel(root)
    new_window.title("DataFrame Display")

    # Create a frame to hold the Treeview and Scrollbars
    frame = ttk.Frame(new_window)
    frame.pack(expand=True, fill='both')

    # Create a Treeview widget in the new window
    tree = ttk.Treeview(frame)

    # Create a vertical Scrollbar and associate it with the tree
    v_scrollbar = ttk.Scrollbar(frame, orient="vertical", command=tree.yview)
    tree.configure(yscrollcommand=v_scrollbar.set)

    # Create a horizontal Scrollbar and associate it with the tree
    h_scrollbar = ttk.Scrollbar(frame, orient="horizontal", command=tree.xview)
    tree.configure(xscrollcommand=h_scrollbar.set)

    # Display the DataFrame using the previously defined function
    display_dataframe(df_to_display, tree)

    # Layout
    tree.grid(row=0, column=0, sticky='nsew')
    v_scrollbar.grid(row=0, column=1, sticky='ns')
    h_scrollbar.grid(row=1, column=0, sticky='ew')

    # Configure frame's row and column weights so it resizes proportionally
    frame.grid_rowconfigure(0, weight=1)
    frame.grid_columnconfigure(0, weight=1)


def additional_action2():
    # Create a new window
    new_window = tk.Toplevel(root)
    new_window.title("DataFrame Display")

    # Create a Treeview widget in the new window
    tree = ttk.Treeview(new_window)

    # Display the DataFrame using the display_dataframe function
    display_dataframe(df_metrics.reset_index(), tree)

    # Layout
    tree.pack(expand=True, fill='both')

def additional_action3():
    # Create a new window
    new_window = tk.Toplevel(root)
    new_window.title("Directed Graph")

    # Load and display the image
    image = Image.open(path + "Directed Graph.png")
    photo = ImageTk.PhotoImage(image)

    # Create a label for the text and image
    label_text = tk.Label(new_window, text="Directed Graph")
    label = tk.Label(new_window, image=photo)

    # This line is crucial to prevent the image from being garbage collected
    label.image = photo

    # Packing the label
    label_text.pack()
    label.pack()


def additional_action4():
    # Create a new window
    new_window = tk.Toplevel(root)
    new_window.title("ًWeighted Graph")

    # Load and display the image
    image = Image.open(path + "Weighted Graph.png")
    photo = ImageTk.PhotoImage(image)

    # Create a label for the text and image
    label_text = tk.Label(new_window, text="ًWeighted Graph")
    label = tk.Label(new_window, image=photo)

    # This line is crucial to prevent the image from being garbage collected
    label.image = photo

    # Packing the label
    label_text.pack()
    label.pack()

In [25]:
# Defining the Global Variables
df_to_display = None
df_metrics = None

# Enter the info
def get_info():
    # Initializing the Global Variable
    global df_to_display, df_metrics
    query = entry_id1.get()
    display = None
    distance = None
    oj_vs_dist = None

    # Choosing the number of records to display
    try:
        display = int(entry_id2.get())
    except ValueError:
        messagebox.showerror("Error", "Enter a valid integer for the number of records you want to show.")
        return

    # Choosing the type of the distance metric
    if entry_id3.get() not in ["1", "2", "3"]:
        messagebox.showerror("Error", "Enter a number between 1 and 3 for the distance metric.")
        return
    else:
        distance = {"1": "Euclidean Distance", "2": "Manhattan Distance", "3": "Cosine Similarity"}[entry_id3.get()]

    # Choosing whether to display the Distances or Original Table
    if entry_id4.get() not in ["1", "2"]:
        messagebox.showerror("Error", "Enter a number between 1 and 2 for the table to be displayed.")
        return
    else:
        oj_vs_dist = {"1": "Distances", "2": "Original"}[entry_id4.get()]

    # Displaying all choices
    messagebox.showinfo("Info", f"\nYour choices:\nQuery: {query}\nNumber of records: {display}\nDistance metric: {distance}\nDisplay option: {oj_vs_dist}")

    # Preprocess and TF-IDF for the query
    query = [clean_text(query)]
    tfidf_query = vectorizer.transform(query)

    # Calculate the Distances
    euclidean_distance = euclidean_distances(tfidf_query, tfidf_documents)
    normalized_euclidean_distance = (euclidean_distance - euclidean_distance.min()) / (euclidean_distance.max() - euclidean_distance.min())
    manhattan_distance = manhattan_distances(tfidf_query, tfidf_documents)
    normalized_manhattan_distance = (manhattan_distance - manhattan_distance.min()) / (manhattan_distance.max() - manhattan_distance.min())
    cosine_distance = cosine_similarity(tfidf_query, tfidf_documents)

    # Select case descriptions that contain at least one word from the query
    query_indices = df[df['CASE_DESC'].apply(lambda desc: any(k in desc for k in set(k for sublist in query for k in sublist.split())))].index.tolist()
    df_oj_results = pd.DataFrame({"CASE_DESC" : df_oj["CASE_DESC"],
              "Euclidean Distance" : normalized_euclidean_distance.flatten(),
              "Manhattan Distance" : normalized_manhattan_distance.flatten(),
              "Cosine Similarity" : cosine_distance.flatten()}).loc[query_indices].sort_index()
    df_temp = df_oj_results.sort_values(by = distance, ascending = False if distance == "Cosine Similarity" else True)
    df_to_display = df_oj.loc[df_temp.index].head(display) if oj_vs_dist == "Original" else df_temp.head(display)
    print(df_to_display)

    # Use the preprocessed case descriptions for the graphs
    def results(distance):
        df_results = pd.DataFrame({"CASE_DESC" : df["CASE_DESC"],
                                "Euclidean Distance" : normalized_euclidean_distance.flatten(),
                                "Manhattan Distance" : normalized_manhattan_distance.flatten(),
                                "Cosine Similarity" : cosine_distance.flatten()}).loc[query_indices].sort_index()
        return df_results.sort_values(by = distance, ascending = False if distance == "Cosine Similarity" else True)
    
    # Directed Graph
    top_case_desc = [" ".join([get_display(arabic_reshaper.reshape(j)) for j in word_tokenize(i)]) for i in list(results(distance)['CASE_DESC'].head(display))]
    def remove_duplicates(sentence):
        # Split the sentence into words
        words = sentence.split()

        # Use an ordered dictionary to remove duplicates while preserving order
        from collections import OrderedDict
        unique_words = OrderedDict.fromkeys(words)

        # Join the unique words back into a sentence
        return ' '.join(unique_words)
    top_case_desc = [remove_duplicates(i) for i in top_case_desc]
    names_dict = {get_display(arabic_reshaper.reshape(j)): j for i in list(results(distance)['CASE_DESC'].head(display)) for j in word_tokenize(i)}
    nodes = list(set(word_tokenize(" ".join(top_case_desc))))
    edges_directed = [item for sublist in [[(i[j], i[j+1]) for j in range(len(i) - 1)] for i in [word_tokenize(i) for i in top_case_desc]] for item in sublist]
    G = nx.DiGraph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges_directed)
    pos = nx.circular_layout(G)
    nx.draw(G, pos, with_labels = True, arrows = True, node_size = 1000, node_color = 'red', font_color = 'white', arrowsize = 10, font_size = 10)
    plt.savefig("Directed Graph.png", format='PNG')
    plt.show(block=False)
    df_metrics = pd.DataFrame({"Page Rank":nx.pagerank(G),
                           "Degree Centrality":nx.degree_centrality(G),
                           "Betweenness Centrality":nx.betweenness_centrality(G),
                           "Closeness Centrality":nx.closeness_centrality(G)})
    df_metrics = df_metrics.rename(index=names_dict)

    # Weighted Graph
    temp_list = [(word, k) for i in top_case_desc for j, word in enumerate(word_tokenize(i)) for k in word_tokenize(i)[j + 1:]]
    for i in [(nodes[i], nodes[j]) for i in range(len(nodes) - 1) for j in range(i + 1, len(nodes))]:
        temp_list.append(i)
    edges_weighted = [(key[0], key[1], value-1) for key, value in dict(pd.Series([tuple(sorted(i)) for i in temp_list]).value_counts()).items()]
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_weighted_edges_from(edges_weighted)
    pos = nx.circular_layout(G)
    nx.draw(G, pos, with_labels = True, arrows = True, node_size = 1000, node_color = 'red', font_color = 'white', font_size = 10)
    nx.draw_networkx_edge_labels(G, pos, nx.get_edge_attributes(G, "weight"))
    plt.savefig("Weighted Graph.png", format='PNG')
    plt.show(block=False)

    # Adding the 4 buttons
    btn_additional1.pack()
    btn_additional2.pack()
    btn_additional3.pack()
    btn_additional4.pack()

# Main window
root = tk.Tk()
root.title("Query System")

# Widgets
label_id1 = tk.Label(root, text="Please enter your search query:")
entry_id1 = tk.Entry(root)
label_id2 = tk.Label(root, text="\nEnter the number of records you want to display:")
entry_id2 = tk.Entry(root)
label_id3_0 = tk.Label(root, text="\nSelect a distance metric for sorting the records:")
label_id3_1 = tk.Label(root, text="1. Euclidean Distance")
label_id3_2 = tk.Label(root, text="2. Manhattan Distance")
label_id3_3 = tk.Label(root, text="3. Cosine Similarity")
label_id3_4 = tk.Label(root, text="Enter the number of your choice:")
entry_id3 = tk.Entry(root)
label_id4_0 = tk.Label(root, text="\nWould you like to display:")
label_id4_1 = tk.Label(root, text="1. The distances table sorted by the chosen distance")
label_id4_2 = tk.Label(root, text="2. The original dataset sorted by the chosen distance")
label_id4_3 = tk.Label(root, text="Enter the number of your choice:")
entry_id4 = tk.Entry(root)
btn_get_info = tk.Button(root, text="Submit", command=get_info)
info_text = tk.Label(root, text="", font=("Helvetica", 12))

# Packing all elements
label_id1.pack()
entry_id1.pack()
label_id2.pack()
entry_id2.pack()
label_id3_0.pack()
label_id3_1.pack()
label_id3_2.pack()
label_id3_3.pack()
label_id3_4.pack()
entry_id3.pack()
label_id4_0.pack()
label_id4_1.pack()
label_id4_2.pack()
label_id4_3.pack()
entry_id4.pack()
btn_get_info.pack()
info_text.pack()

# Additional Buttons Definition
btn_additional1 = tk.Button(root, text="Display DataFrame", command=additional_action1)
btn_additional2 = tk.Button(root, text="Display Centrality Measures", command=additional_action2)
btn_additional3 = tk.Button(root, text="Display Directed Graph", command=additional_action3)
btn_additional4 = tk.Button(root, text="Display Weighted Graph", command=additional_action4)

# Set the additional buttons to be initially hidden
btn_additional1.pack_forget()
btn_additional2.pack_forget()
btn_additional3.pack_forget()
btn_additional4.pack_forget()

root.mainloop()